<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/Eye_ai_new_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Prerequisites
!pip install --upgrade --force pydantic git+https://github.com/fair-research/bdbag git+https://github.com/informatics-isi-edu/deriva-py git+https://github.com/informatics-isi-edu/eye-ai-tools git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
import json
import os
from eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
import torch


In [ ]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = False #@param ["False", "True"] {type:"raw"}
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


In [ ]:
# change cache_dir and working_dir
cache_dir = 'content/cache/'
working_dir = 'content/working/'

EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)


In [ ]:
# @title Initiate an Execution
# configuration_rid="2-A5JP" : 10% of subjects for train, validation, and test set
# configuration_rid="2-A5JR"   Full set of subjects for train, validation, and test set
configuration_records = EA.execution_init(configuration_rid="2-A5JP")
configuration_records.model_dump()

In [ ]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
train_dir = configuration_records.bag_paths[0]
validation_dir = configuration_records.bag_paths[1]
test_dir = configuration_records.bag_paths[2]

train_cropped_image_path, train_cropped_csv = EA.create_cropped_images(str(train_dir), output_dir = working_dir, crop_to_eye=False)
validation_cropped_image_path, validation_cropped_csv = EA.create_cropped_images(str(validation_dir), output_dir = working_dir, crop_to_eye=False)
test_cropped_image_path, test_cropped_csv = EA.create_cropped_images(str(test_dir), output_dir = working_dir, crop_to_eye=False)


In [ ]:
# @title Execute Proecss algorithm (Cropping)
from eye_ai_ml.glaucoma.optic_disk_crop import preprocess_and_crop
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  model = torch.load(PATH)

In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)